In [1]:
import numpy as np, pandas as pd, json, glob, os, chromadb, openai
# from sentence_transformers import SentenceTransformer
from openai import OpenAI
from typing import List, Tuple, Union
from langchain_ollama.embeddings import OllamaEmbeddings
from tqdm.auto import tqdm
import shutil
from chromadb import Documents, EmbeddingFunction, Embeddings


/home/jellybee/anaconda3/envs/llm_311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
key = open('../../../../api_key.txt','r')
api_key = key.read()
openai.api_key = api_key

base_ = open('../../../../base_url.txt','r')
base_url = base_.read()
# openai.api_key = api_key

In [3]:
# class OllamaSentenceTransformer():
#     def __init__(
#             self,
#             *args,
#             **kargs,
#             ) -> None:
                
#                 self.model = kargs.get("model","bge-m3")
#                 self.embedding_model = OllamaEmbeddings(
#                             model = self.model,

#                         )
                        
    
#     def encode(
#             self,
#             documents:Union[str, List[str], np.ndarray],
#             *args,
#             **kargs,
#         )-> np.ndarray:
#         if isinstance(documents, str):
#             document_embeddings = self.embedding_model.embed_query(documents)
#             return np.array(document_embeddings)
        
#         if isinstance(documents, np.ndarray):
#             documents = documents.tolist()
        
#         document_embeddings = [self.embedding_model.embed_query(s) for s in documents]
#         return np.array(document_embeddings)

# sentence_transformer = OllamaSentenceTransformer()


In [4]:
# class OllamaSentenceTransformer(EmbeddingFunction):
#     def __init__(
#             self,
#             *args,
#             **kargs,
#             ) -> Embeddings:
#                 # self.base_url = kargs.get("base_url", "http://localhost:11434")
#                 self.model = kargs.get("model","bge-m3")
#                 self.embedding_model = OllamaEmbeddings(
#                             model = self.model,
#                             # base_url = base_url,
#                         )
                        
    
#     def encode(
#             self,
#             documents:Union[str, List[str], np.ndarray],
#             *args,
#             **kargs,
#         )-> np.ndarray:
#         if isinstance(documents, str):
#             document_embeddings = self.embedding_model.embed_query(documents)
#             return np.array(document_embeddings)
        
#         if isinstance(documents, np.ndarray):
#             documents = documents.tolist()
        
#         document_embeddings = [self.embedding_model.embed_query(s) for s in documents]
#         return np.array(document_embeddings)

# sentence_transformer = OllamaSentenceTransformer()


In [5]:
# from chromadb import Documents, EmbeddingFunction, Embeddings

# class MyEmbeddingFunction(EmbeddingFunction):
#     def __init__(self, input: Documents) -> Embeddings:
#         # embed the documents somehow
#         # self.model = kargs.get("model","bge-m3")
#         self.embedding_model = OllamaEmbeddings(
#                             model = 'bge-m3',)
        
        
        
#         return embeddings

In [6]:
class MyEmbeddingFunction(EmbeddingFunction):
    def __init__(self, model: str = "bge-m3"):
        self.model = model
        self.embedding_model = OllamaEmbeddings(model=self.model)

    def __call__(self, input: Documents) -> Embeddings:
        # input은 List[str] 형태로 들어옴
        if isinstance(input, str):
            input = [input]

        embeddings = [self.embedding_model.embed_query(doc) for doc in input]
        return np.array(embeddings).tolist()  # ChromaDB는 List[List[float]] 기대
embedding_fn = MyEmbeddingFunction(model="bge-m3")


In [7]:
# embedding_fn

In [8]:
db_path = "../data/chroma_db"
if os.path.exists(db_path):
    shutil.rmtree(db_path)
    print("기존 ChromaDB 삭제 완료")

In [9]:
# from chromadb.utils import embedding_functions

# bge_embed = embedding_functions.SentenceTransformerEmbeddingFunction(
#     model_name="BAAI/bge-m3"
# )

In [10]:
# ChromaDB 클라이언트
# db_path = "../../../../data/chroma_db"
client = chromadb.PersistentClient(path =db_path)

#collection 삭제
# client.delete_collection('medical_qa')

# Collection 생성
collection = client.get_or_create_collection('medical_qa',
                                             embedding_function=embedding_fn
                                             )


In [11]:
'../../../medical_data/02.라벨링데이터/'

'../../../medical_data/02.라벨링데이터/'

In [11]:
path = '../../../medical_data/02.라벨링데이터/'
for domain in os.listdir(path):
    print(domain.split('/')[-1])
    for file in tqdm(os.listdir(os.path.join(path, domain))[:3]):
        file_path = os.path.join(path, domain, file)
        with open(file_path, 'r', encoding='utf-8-sig') as f:
            json_file = f.read()
            data = json.loads(json_file)

        print(data)
        question = data.get("question" , "").strip()
        # print(question)
    # break
        # embedding = embedding_fn(question)
        # embedding = embedding.astype('float64').tolist()
        # print(embedding)
        # break
    # break
        # embedding = embedding.astype(float).tolist()
        # # print(embedding)
        collection.add(
            ids = [file],
            documents=[question],
            # embeddings = embedding,
            metadatas = [{
                "answer" : data['answer'],
                "qa_id" : data["qa_id"],
                "domain" : data["domain"],
                "q_type" : data["q_type"],
            }],
        )     
        # collection.update(
        #     ids = [file],
        #     # documents=[question],
        #     embeddings = [embedding],
            
        # )   

        # break
    # break
        
    

TL_내과


  0%|          | 0/3 [00:00<?, ?it/s]

{'qa_id': 629424, 'domain': 17, 'q_type': 1, 'question': "67세 여성이 우상복부 통증을 주소로 내원하였다. 통증은 간헐적으로 발생하며, 특히 기름진 음식을 섭취한 후 더 심해진다고 한다. 환자는 최근 피로감과 미열을 동반하였으며, 체중 감소를 호소하였다. 신체검사에서 우상복부 압통이 관찰되었고, Murphy's sign이 양성이었다. 이 환자의 진단을 위해 가장 적절한 첫 번째 영상 검사는 무엇인가?  \n1) 복부 초음파  \n2) 복부 X-ray  \n3) 혈청 아밀라아제 검사  \n4) 대변 잠혈 검사  \n5) 상부 위장관 내시경", 'answer': '1) 복부 초음파'}


 33%|███▎      | 1/3 [00:00<00:01,  1.98it/s]

{'qa_id': 615373, 'domain': 17, 'q_type': 1, 'question': '유전성 혈색소 침착증의 주요 치료 방법 중 하나로, 과도한 철분을 제거하기 위해 시행되는 치료는 무엇인가요?  \n1) 방사선 치료  \n2) 화학 요법  \n3) 정맥 절개술(사혈 치료) \n4) 면역 요법  \n5) 항생제 투여', 'answer': '3) 정맥 절개술(사혈 치료)'}


...

 33%|███▎      | 1/3 [00:00<00:00,  5.63it/s]

{'qa_id': 620175, 'domain': 15, 'q_type': 2, 'question': '신생아 선별검사를 통해 조기에 발견하지 못하면 지능 저하와 같은 돌이킬 수 없는 손상이 발생할 수 있는 내분비 질환은 무엇인가?', 'answer': '선천성 갑상선 기능 저하증'}
{'qa_id': 21009, 'domain': 15, 'q_type': 1, 'question': '8세 남아가 경련을 주소로 내원하였다. 10일 전 고열, 인두통, 구토가 있었고, 하루 뒤 목, 가슴 윗부분 및 허벅지 부위에 발진이 나타났으나 해열제 치료 후 5일 만에 소실되었다. 내원 전날 소변량 감소, 콜라색 소변, 안면 부종이 발생하였다. 이 환아에서 가장 시급히 시행해야 할 처치는 무엇인가?  \n1) 혈압을 측정한다.  \n2) 스테로이드를 투여한다.  \n3) 알부민을 정맥 주사한다.  \n4) 항생제를 투여한다.  \n5) 생리식염수를 정맥 주사한다.', 'answer': '1) 혈압을 측정한다.'}


 67%|██████▋   | 2/3 [00:00<00:00,  2.91it/s]

{'qa_id': 62, 'domain': 15, 'q_type': 1, 'question': "신생아 중환자실에서 기계환기를 받고 있는 3주 된 미숙아가 폐 분비물 제거를 위해 치료를 받고 있다. 다음 중 폐 분비물 제거를 위한 흉부 물리치료(CPT) 기법으로 권장되지 않는 것은 무엇인가?  \n1) 자세배액, 타진, 진동(Postural Drainage, Percussion, and Vibration) \n2) 흡인(Suction) 단독  \n3) 발꿈치 접촉 타진(Contact Heel Percussion) \n4) '컵핑' 타진(Cupping Percussion) \n5) 폐 짜기 기법(Lung Squeezing Technique)", 'answer': '5) 폐 짜기 기법(Lung Squeezing Technique)'}


100%|██████████| 3/3 [00:01<00:00,  2.79it/s]


In [15]:
collection.query(
    query_texts=["신생아 중환자실에서 기계환기를 받고 있는"],
    n_results=3,
    where={"domain":15}
    
    
)

{'ids': [['필수_62.json', '필수_21009.json', '필수_620175.json']],
 'embeddings': None,
 'documents': [["신생아 중환자실에서 기계환기를 받고 있는 3주 된 미숙아가 폐 분비물 제거를 위해 치료를 받고 있다. 다음 중 폐 분비물 제거를 위한 흉부 물리치료(CPT) 기법으로 권장되지 않는 것은 무엇인가?  \n1) 자세배액, 타진, 진동(Postural Drainage, Percussion, and Vibration) \n2) 흡인(Suction) 단독  \n3) 발꿈치 접촉 타진(Contact Heel Percussion) \n4) '컵핑' 타진(Cupping Percussion) \n5) 폐 짜기 기법(Lung Squeezing Technique)",
   '8세 남아가 경련을 주소로 내원하였다. 10일 전 고열, 인두통, 구토가 있었고, 하루 뒤 목, 가슴 윗부분 및 허벅지 부위에 발진이 나타났으나 해열제 치료 후 5일 만에 소실되었다. 내원 전날 소변량 감소, 콜라색 소변, 안면 부종이 발생하였다. 이 환아에서 가장 시급히 시행해야 할 처치는 무엇인가?  \n1) 혈압을 측정한다.  \n2) 스테로이드를 투여한다.  \n3) 알부민을 정맥 주사한다.  \n4) 항생제를 투여한다.  \n5) 생리식염수를 정맥 주사한다.',
   '신생아 선별검사를 통해 조기에 발견하지 못하면 지능 저하와 같은 돌이킬 수 없는 손상이 발생할 수 있는 내분비 질환은 무엇인가?']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'answer': '5) 폐 짜기 기법(Lung Squeezing Technique)',
    'q_type': 1,
    'qa_id': 62,
    'domain': 15},
   {'qa_id': 21009, 'q_type': 1, 'dom

In [12]:
all_doc = collection.get()

In [ ]:
all_doc['']

{'ids': ['필수_629424.json',
  '필수_615373.json',
  '필수_612657.json',
  '필수_625882.json',
  '필수_617405.json',
  '필수_3736.json',
  '필수_627611.json',
  '필수_625552.json',
  '필수_629499.json',
  '필수_620175.json',
  '필수_21009.json',
  '필수_62.json'],
 'embeddings': None,
 'documents': ["67세 여성이 우상복부 통증을 주소로 내원하였다. 통증은 간헐적으로 발생하며, 특히 기름진 음식을 섭취한 후 더 심해진다고 한다. 환자는 최근 피로감과 미열을 동반하였으며, 체중 감소를 호소하였다. 신체검사에서 우상복부 압통이 관찰되었고, Murphy's sign이 양성이었다. 이 환자의 진단을 위해 가장 적절한 첫 번째 영상 검사는 무엇인가?  \n1) 복부 초음파  \n2) 복부 X-ray  \n3) 혈청 아밀라아제 검사  \n4) 대변 잠혈 검사  \n5) 상부 위장관 내시경",
  '유전성 혈색소 침착증의 주요 치료 방법 중 하나로, 과도한 철분을 제거하기 위해 시행되는 치료는 무엇인가요?  \n1) 방사선 치료  \n2) 화학 요법  \n3) 정맥 절개술(사혈 치료) \n4) 면역 요법  \n5) 항생제 투여',
  '경증에서 중등증의 COVID-19 환자에서 이버멕틴의 사용이 사망률에 미치는 영향을 평가한 연구 결과는 무엇인가?\n\n1) 사망률이 유의미하게 감소한다.  \n2) 사망률이 유의미하게 증가한다.  \n3) 사망률에 유의미한 변화가 없다.  \n4) 사망률이 약간 감소한다.  \n5) 사망률이 약간 증가한다.',
  '58세 남성이 갑작스러운 흉통을 호소하며 응급실에 내원하였다. 심전도 검사에서 ST 분절 상승이 확인되었으며, 심근경색이 의심된다. 환자는 10년 전부터 고혈압 약물을 복용 중이며, 최근에는 고지혈증으로 인해 스타틴을 추가로 복용하고 